In [ ]:
import re
import json
from langchain_community.chat_models import ChatOllama 
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.embeddings import OllamaEmbeddings
from langchain.agents import initialize_agent, AgentType
import chardet
from pathlib import Path
from typing import List
from langchain.docstore.document import Document as LangDocument
from ragFunction import * 

# 單個檔案測試
MASTER_JS_PATH = r"C:\users\tuf\documents\cloudysys_nickfury\dcmsln\dcmsln_201812\Master\PageJS\Query\MG0900S.js"


In [50]:
# Json 輸出格式解析器
parser = JsonOutputParser()
format_instructions = parser.get_format_instructions()

# 使用本地 LLM 模型
embed_model = OllamaEmbeddings(model="nomic-embed-text") 
fst_llm = ChatOllama(model="qwen3:8b") #提出
sec_llm = ChatOllama(model="gemma3:4b") #審查
third_llm = ChatOllama(model="deepcoder:1.5b") #檢查改動是否符合原始邏輯，若符合則發動投票

In [ ]:
# 提示詞
# Agent 1：提出修改方案
fst_prompt = PromptTemplate.from_template("""
You are a JavaScript static analysis engine. Your job is to detect outdated, deprecated, or bad practice code from the following JavaScript function snippet, and refactor it to modern best-practice syntax.

Analyze each function thoroughly. If no improvement is needed, state so clearly. Otherwise, return both the original (worse) and the improved (better) versions.

When given the code, proceed as follows:

Question: Please improve the following JavaScript code function:
{code}.
Your Final Answer MUST following format:
{{
  "worse": <original_function>,
  "better": <refactored_function>
}}
""")

# Agent 2：檢查提案
sec_prompt = PromptTemplate.from_template("""
You are a senior JavaScript code reviewer with deep experience in modern best practices and refactoring strategies.
You will be shown a proposal that compares a refactored function with its original version.
Your task is to carefully evaluate the changes and vote whether the proposal is acceptable or not based on clarity, correctness, performance, and adherence to modern JavaScript practices.

Analyze the input proposal **line by line**, and consider if the suggested code improves readability, maintainability, and follows modern conventions.

=== Proposal ===
{proposal}
===

Question: Is this refactoring acceptable? Your Final Answer MUST following format:
{{
    "vote": "approve" | "disapprove",
    "reason": "your explanation here"
}}
""")

# Agent 3：檢查語意一致
third_prompt = PromptTemplate.from_template("""
You are a JavaScript semantic consistency reviewer.
Your job is to verify if the REWRITTEN JavaScript code maintains the exact same **logic and behavior** as the ORIGINAL code.

=== ORIGINAL CODE ===
{original}

=== REWRITTEN CODE ===
{rewritten}

Question: Does the rewritten code preserve the full intent and behavior of the original? Your Final Answer MUST following format:
{{
    "vote": "approve" | "disapprove",
    "reason": "your explanation here"
}}
""")

In [52]:
# 建立給agent使用的工具

from langchain.tools import Tool
from json_repair import repair_json

def extract_json(text: str) -> str:
    try:
        start = text.find('[')
        end = text.rfind(']') + 1
        if start == -1 or end <= start:
            return "No JSON array found."
        json_str = text[start:end]
        json_str = json_str.encode('utf-8').decode('unicode_escape')
        repaired_string = repair_json(json_str)
        try:
            parsed = json.loads(repaired_string)
        except json.JSONDecodeError:
            cleaned = re.sub(r',\s*([\]}])', r'\1', repaired_string)
            repaired_string = repair_json(cleaned)
            parsed = json.loads(repaired_string)
        return json.dumps(parsed, ensure_ascii=False, indent=2)
    except Exception as e:
        return f"Failed to extract JSON: {str(e)}"


tools = [
    Tool(
        name="check json format",
        func=extract_json,
        description="Use this tool when you receive a string that may contain a JSON array and you need to extract or validate it."
    )
]

In [53]:
# 建立 Agent
agent_1 = initialize_agent(
    tools=tools,
    llm=fst_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

agent_2 = initialize_agent(
    tools=tools,
    llm=sec_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

agent_3 = initialize_agent(
    tools=tools,
    llm=third_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

In [54]:
# 呼叫LLM
def call_llm(llm, prompt):
    response = llm.invoke(prompt)
    return response.content if hasattr(response, "content") else str(response)

In [55]:
# 建立 decoder 讀檔案
def read_text_file_safely(path: str) -> str:
    with open(path, "rb") as f:
        raw_data = f.read()
        detected = chardet.detect(raw_data)
        encoding = detected["encoding"] or "utf-8"

    fallback_encodings = []

    if encoding: # Add the detected encoding first
        fallback_encodings.append(encoding)
    if 'GB2312' not in fallback_encodings: # Add GB2312 if not already present
        fallback_encodings.append("GB2312")

    # Ensure common encodings are covered
    additional_encodings = ["utf-8", "big5", "cp950", "gbk", "gb18030", "utf-16", "windows-1252"]
    for enc in additional_encodings:
        if enc not in fallback_encodings:
            fallback_encodings.append(enc)

    for enc in fallback_encodings:
        try:
            print(f"嘗試使用編碼decode: {enc}")
            return raw_data.decode(enc)
        except UnicodeDecodeError:
            continue

    raise UnicodeDecodeError(f"嘗試的編碼都無法decode: {path}")

In [56]:
# 主流程
def code_rewrite_proposal(code: str) -> str:
    # Agent 1：提案改寫
    fst_input = fst_prompt.format(code=code)
    # fst_response = call_llm(fst_llm, fst_input)
    # 改成agent with tool
    fst_response = agent_1.run(fst_input)
    print(fst_response)
    return fst_response

def code_linting_report(proposals: str) -> List:
    if not proposals:
        return [{"status": "no-change", "message": ""}]
    
    try:
        proposals_list = json.loads(proposals)
    except json.JSONDecodeError as e:
        print(f"Invalid JSON input: {e}")
        return [{"status": "invalid-json", "message": str(e)}]
    
    sec_response_list = []

    for i, p in enumerate(proposals_list):
        print(f"\nProcessing proposal {i+1}: {p}")
        # 檢查關鍵鍵是否存在
        if not all(k in p for k in ["line", "original", "issue", "replacement", "reason"]):
            print(f"Skipping proposal {i+1} due to missing required keys: {p}")
            continue

        sec_input = sec_prompt.format(proposal=json.dumps([p], ensure_ascii=False))
        sec_response = agent_2.run(sec_input)
        print(sec_response)
        sec_response_list.append(sec_response)

    return sec_response_list

In [ ]:
# 單一檔案測試

def run_first_js_review(folder_path: str):

    code = read_text_file_safely(folder_path)
    # function_div = extract_js_function_blocks(code)
    result = code_rewrite_proposal(code)
    print(result)


run_first_js_review(MASTER_JS_PATH)

嘗試使用編碼decode: UTF-8-SIG


> Entering new AgentExecutor chain...
<think>
Okay, let's tackle this problem. The user provided a JavaScript code snippet from a VB.NET file, but the code seems to be JavaScript. The task is to refactor it into modern best practices.

First, I need to understand the code. The functions are part of a web application, probably using ASP.NET Web Forms since there are elements like document.all and webForm. The code has several functions, including PageSpecialInit, bPageChk, Distinguish, fillReturnInfo, checkComparisonProdID, and checkProdID.

Looking at the code, there are a few issues. For example, in the bPageChk function, the switch statement uses string comparisons with case ("LINKBUTTON"), which is okay, but maybe using const variables for the case values could improve readability. Also, the use of document.all is deprecated; modern JavaScript uses document.getElementById or querySelector.

Another thing is the use of event.returnValue and event.keyCode. Th

KeyboardInterrupt: 